# Generating Irony data

Results for *"Qwen/Qwen2.5-0.5B-Instruct"* and *"deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"*:
- irony seems a too difficult topic for this models, the "irony" sentences aren't ironic (also the "targeted" prompts case).
- doesn't follow always the instruction and is very sensitive to prompt, but it follows nicely the JSON format.

In [1]:
import os

# Move up one directory
if os.path.basename(os.getcwd()) == "mycode":
    os.chdir("..")
    
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import re
import json
import time
from mycode.utilities import log_synthetic_data, response2json, get_response

# file where the logs will be saved
log_file_path = "synthetic_data/semevalirony_log.json"
os.remove(log_file_path)

folder_name = "synthetic_data"
os.makedirs(folder_name, exist_ok=True) # Create folder if it doesn't exist

df = pd.read_csv("LREC-COLING/train/semevalironytrainAll.csv")
df = df.rename(columns={"1": "text", "2": "label"})
display(df.head())
first_irony = df[df['label'] == 'irony'].iloc[0].loc['text']
first_non_irony = df[df['label'] == 'non irony'].iloc[0].loc['text']

print("\nFirst irony example:")
print(first_irony)

print("\nFirst non irony example:")
print(first_non_irony)

,0,text,label
0,0,seeing ppl walking w/ crutches makes me really...,irony
1,1,"look for the girl with the broken smile, ask h...",non irony
2,2,Now I remember why I buy books online @user #s...,irony
3,3,@user @user So is he banded from wearing the c...,irony
4,4,Just found out there are Etch A Sketch apps. ...,irony



First irony example:
seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life

First non irony example:
look for the girl with the broken smile, ask her if she wants to stay while, and she will be loved. 💕🎵


## Qwen/Qwen2.5-0.5B-Instruct
### Baseline

In [2]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda",
    attn_implementation='flash_attention_2',
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# for qwen we use this system prompt:
system_prompt_qwen = "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."

In [3]:
prompt ="""\
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non irony". \
Produce 5 examples  for "irony" and 5 examples for "non irony". 

Use this format for the examples:
text: <text>
label: <label>\
"""

generated, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 8.7
GENERATED RESPONSE:
Sure! Here are 10 examples of detecting irony in written text:

### Irony Examples

**Example 1:**  
Text: "He was so hungry he could barely stand up."
Label: **Irony**

**Explanation:** The sentence describes a person who is already very hungry but can't even stand up due to the severity of their hunger.

**Example 2:**  
Text: "She always said she would come tomorrow, but I never heard from her again."
Label: **Non-Irony**

**Explanation:** This statement implies that she has been completely absent from communication since the previous day, which contradicts the claim made about coming back.

**Example 3:**  
Text: "The book was so boring it was hard to put down."
Label: **Irony**

**Explanation:** The phrase "so boring" suggests that the content of the book is uninteresting or tedious, making it difficult to stay engaged.

**Example 4:**  
Text: "He's not going anywhere; he just needs a change of scenery."
Label: **Irony**

**Explanation:** This i

In [4]:
# With example for the two classes
prompt =f"""\
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non irony". \
Produce exactly 5 examples  for "irony" and 5 examples for "non irony". 

Use this format for generating the data:
text: {first_irony}
label: irony

text: {first_non_irony}
label: non irony\
"""

generated, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 0.79
GENERATED RESPONSE:
text: seeing people with wheelchairs makes me feel very fulfilled for the next three months.
label: irony

text: I'm so excited because there's a new dress that looks amazing on me. It's just like having a whole new outfit!
label: non-irony


In [5]:
# act as a linguist and NLP practitioner
prompt =f"""\
You are an expert linguist and NLP practitioner specializing in irony detection. \
Your task is to generate high-quality synthetic examples of ironic and non-ironic text to improve an irony detection model

Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non irony". \
Produce exactly 5 examples for "irony" and 5 examples for "non irony". 

Use this format for generating the data:
text: <text>
label: <label>\
"""

generated, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 5.02
GENERATED RESPONSE:
Sure! I'll provide you with five examples of irony and five examples of non-irony text. Here's how they can be structured:

### Irony Example 1
**Text:** The company's new product line was designed with cutting-edge technology that would revolutionize the market.
**Label:** Irony

### Irony Example 2
**Text:** She told me she had been diagnosed with a rare disease, but her symptoms were mild.
**Label:** Irony

### Non-Irony Example 3
**Text:** He said he would never give up on his dream job, even if it meant sacrificing his family life.
**Label:** Non-Irony

### Irony Example 4
**Text:** The politician promised to reduce taxes, but instead increased them.
**Label:** Irony

### Irony Example 5
**Text:** The student confidently answered every question, but then admitted they were wrong.
**Label:** Irony

### Non-Irony Example 6
**Text:** The restaurant staff was very friendly and helpful, but their service was slow.
**Label:** Non-Irony

### Irony Exa

In [ ]:
prompt = f"""\
You are an expert linguist and NLP practitioner specializing in irony detection. \
Your task is to generate **10 high-quality examples** of "ironic" and non-ironic statements, \
with **5 ironic** and **5 non-ironic** examples across different contexts.

### **Output Format (JSON)**
Return **only** a valid JSON list in the following structure:

[
    {{"text": "{first_irony}", "label": "ironic"}},
    {{"text": "{first_non_irony}", "label": "non-ironic"}},
    ...
]
"""

response, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 5.42
GENERATED RESPONSE:
```json
[
    {
        "text": "I'm so happy that I'm getting more money every day! 🚀💸",
        "label": "ironic"
    },
    {
        "text": "It's raining cats and dogs outside today, but I still want to go out. ☠️💨",
        "label": "ironic"
    },
    {
        "text": "The new kid at school seems like such a fun person, even though they're not as outgoing as you might think. 🤔👥",
        "label": "ironic"
    },
    {
        "text": "I've been working on this project all day long, but I just can't seem to get it done. 😞😢",
        "label": "ironic"
    },
    {
        "text": "The weather report says it's going to rain tomorrow, but I'm already planning on going out. 🌡️🌧️",
        "label": "non-ironic"
    },
    {
        "text": "Everyone has their own opinion about this movie, and I don't care what anyone else thinks. 😊👍",
        "label": "non-ironic"
    },
    {
        "text": "The cat was sleeping peacefully inside the house, but 

In [7]:
synthetic_data = response2json(response)

print("synthetic_data[:2]: ", synthetic_data[:2])

synthetic_data[:2]:  [{'text': "I'm so happy that I'm getting more money every day! 🚀💸", 'label': 'ironic'}, {'text': "It's raining cats and dogs outside today, but I still want to go out. ☠️💨", 'label': 'ironic'}]


In [8]:
log_synthetic_data(model_name, "baseline", prompt, synthetic_data, delta_t, output_file=log_file_path)

# Load JSON file
with open(log_file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Print the first log entry
print(json.dumps(data[:1], indent=4, ensure_ascii=False))  # Pretty-print the first entry

Logged 9 examples to synthetic_data/semevalirony_log.json. Time taken: 5.42 seconds
[
    {
        "timestamp": "2025-03-07T10:45:17.727523",
        "model": "Qwen/Qwen2.5-0.5B-Instruct",
        "generation_method": "baseline",
        "prompt": "You are an expert linguist and NLP practitioner specializing in irony detection. Your task is to generate **10 high-quality examples** of ironic and non-ironic statements, with **5 ironic** and **5 non-ironic** examples across different contexts.\n\n### **Output Format (JSON)**\nReturn **only** a valid JSON list in the following structure:\n\n[\n    {\"text\": \"seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life\", \"label\": \"ironic\"},\n    {\"text\": \"look for the girl with the broken smile, ask her if she wants to stay while, and she will be loved. 💕🎵\", \"label\": \"non-ironic\"},\n    ...\n]\n",
        "time_taken_seconds": 5.42,
        "num_examples": 9,
        "generated_examples": [
        

In [9]:
# Convert log entries to a DataFrame
df = pd.DataFrame(data)

# Expand the 'generated_examples' column
samples_df = df.explode("generated_examples").reset_index(drop=True)

# Convert 'generated_examples' (which is still a dictionary) into separate columns
samples_df = pd.concat([samples_df.drop(columns=["generated_examples"]), samples_df["generated_examples"].apply(pd.Series)], axis=1)

display(samples_df.head())

,timestamp,model,generation_method,prompt,time_taken_seconds,num_examples,text,label
0,2025-03-07T10:45:17.727523,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.42,9,I'm so happy that I'm getting more money every...,ironic
1,2025-03-07T10:45:17.727523,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.42,9,"It's raining cats and dogs outside today, but ...",ironic
2,2025-03-07T10:45:17.727523,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.42,9,The new kid at school seems like such a fun pe...,ironic
3,2025-03-07T10:45:17.727523,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.42,9,I've been working on this project all day long...,ironic
4,2025-03-07T10:45:17.727523,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,5.42,9,The weather report says it's going to rain tom...,non-ironic


### Targeted synthetic data

In [10]:
prompt = f"""\
You are an expert linguist and NLP practitioner. \
List all key linguistic and semantic phenomena that must be covered by a irony detection model to perform accurately. \
Provide concrete examples illustrating each phenomenon. \
"""

response, delta_t_0 = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 10.1
GENERATED RESPONSE:
Certainly! I'd be happy to help you understand the key linguistic and semantic phenomena necessary for a successful irony detection model. Let's break down the components and provide concrete examples:

### 1. **Contextual Awareness**
- **Example**: In a story where someone says "The sun is shining" but later mentions it was raining, this indicates that the irony might have been subtle or ambiguous.
- **Explanation**: The context of weather conditions (sun vs. rain) could significantly alter the meaning and tone of the statement.

### 2. **Sentiment Analysis**
- **Example**: A movie review criticizes the protagonist's character but then describes them as having "unusual courage" at the end, indicating sarcasm.
- **Explanation**: The reviewer’s emotional state ("criticizing") and subsequent description ("unusual courage") can lead to a misinterpretation of the film's sentiment.

### 3. **Wordplay and Irony Structure**
- **Example**: In the sentence "

In [11]:
prompt = f"""\
You are an expert linguist and NLP practitioner. \
List all key linguistic and semantic phenomena that must be considered for an irony detection model to perform accurately. \
Provide only the names of these phenomena as a structured list, without explanations. \
For example: sarcasm, negation, pragmatic inference, unexpected contrast. \
Return the list in a simple bullet-point format.\
"""

response, delta_t_0 = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 0.57
GENERATED RESPONSE:
- Sarcasm
- Negation
- Pragmatic Inference
- Unexpected Contrast
- Wordplay
- Paradox
- Irony
- Metaphor
- Rhetorical Question


In [12]:
# Replace "- " at the beginning of each line with a comma
csv_string = re.sub(r"^\s*-\s*", "", response, flags=re.MULTILINE)  # Remove bullet points
csv_string = ", ".join(csv_string.strip().split("\n"))  # Join lines with ", "

prompt = f"""\
Generate 10 realistic sentences illustrating irony detection examples involving {csv_string}. Ensure that:
- 5 sentences are **ironic**.
- 5 sentences are **non-ironic**.

### **Output Format (JSON)**
Return **only** a valid JSON list in the following structure:

[
    {{"text": "{first_irony}", "label": "ironic"}},
    {{"text": "{first_non_irony}", "label": "non-ironic"}},
    ...
]
"""

response, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)
delta_t += delta_t_0 # Add the time taken for the previous response

# add to log
synthetic_data = response2json(response)
log_synthetic_data(model_name, "targeted", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 4.58
GENERATED RESPONSE:
```json
[
    {
        "text": "I'm so happy because I got a new job right after my last one. 🤩💼",
        "label": "ironic"
    },
    {
        "text": "The best part is when I get to work every day without feeling overwhelmed. 😎💼",
        "label": "ironic"
    },
    {
        "text": "It's hard not to feel jealous sometimes, especially when you're surrounded by friends who love you. 😁🌟",
        "label": "non-ironic"
    },
    {
        "text": "Every time I see someone with a smile on their face, it makes me think of how much they've accomplished. 🌟💡",
        "label": "non-ironic"
    },
    {
        "text": "I'm so grateful for the people around me who support me through thick and thin. 🙏👥",
        "label": "ironic"
    },
    {
        "text": "When I'm working out, I don't even notice how tired I am until it's too late. 🧘‍♀️🏋️‍♂️",
        "label": "non-ironic"
    },
    {
        "text": "The only way to truly understand someone is t

---

### deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B

In [13]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation='flash_attention_2',
)
model.generation_config.pad_token_id = tokenizer.pad_token_id

tokenizer = AutoTokenizer.from_pretrained(model_name)

# we don't submit a system prompt as suggested in the model card

In [14]:
prompt ="""\
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non irony". \
Produce 5 examples  for "irony" and 5 examples for "non irony". 

Use this format for the examples:
text: <text>
label: <label>\
"""

generated, delta_t = get_response(prompt, model, tokenizer, max_new_tokens=2048) # increase max_new_tokens to 1024 for deepseek

TIME TAKEN: 34.33
GENERATED RESPONSE:
Alright, so I need to come up with 10 examples of detecting irony in written text, split evenly into 5 irony examples and 5 non-irony examples. Let me start by understanding what irony is. Irony is when the meaning of something is reversed or contradicted by its execution. It can be direct, indirect, or symbolic.

First, I should think about different contexts where irony is commonly used. Maybe in situations involving people's actions, statements, or situations. I can think of famous examples, like "I never miss a party" from "The Adventures of Huckleberry Finn" because the speaker is actually attending the party. That's a good one for irony.

Another classic example is when someone says, "I'm sorry, but I can't do that" and then goes on to do it anyway. That's a direct example of irony. Maybe something from literature, like Shakespeare's "Hamlet," where the character's actions contradict his words. That's a good non-irony example.

I should also 

In [15]:
# With example for the two classes
prompt =f"""\
Your task is to generate high-quality synthetic examples of ironic and non-ironic text to improve an irony detection model. \
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non irony". \
Produce exactly 5 examples  for "irony" and 5 examples for "non irony". 

Use this format for generating the data:
text: {first_irony}
label: irony

text: {first_non_irony}
label: non irony\
"""

generated, delta_t = get_response(prompt, model, tokenizer, max_new_tokens=2048)

TIME TAKEN: 35.25
GENERATED RESPONSE:
Alright, so I'm trying to help generate some synthetic examples for improving an irony detection model. The user has given me a specific format where I need to produce exactly 5 examples of irony and 5 of non-ironic text. Each example should be in the format: text followed by label, either "irony" or "non-ironic."

First, I need to understand what makes text irony. Irony typically involves a contradiction or imbalance of expectations. It's when the meaning of one part of the sentence is the opposite of another. For example, "I'm so hungry I'll wait till I'm sick to eat" is ironic because the expectation of quick food is contradicted by the slow eating.

I should focus on creating both ironic and non-ironic sentences. For irony, I'll need to craft statements where the action or sentiment doesn't align with the situation. Maybe use phrases that suggest a negative outcome when the positive is expected, or a positive outcome when the negative is expect

In [16]:
# act as a linguist and NLP practitioner
prompt =f"""\
<context>
You are an expert linguist and NLP practitioner specializing in irony detection.
</context>

Your task is to generate high-quality synthetic examples of ironic and non-ironic text to improve an irony detection model. \
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non irony". \
Produce exactly 5 examples for "irony" and 5 examples for "non irony". 

Use this format for generating the data:
text: <text>
label: <label>\
"""

generated, delta_t = get_response(prompt, model, tokenizer, max_new_tokens=2048)

TIME TAKEN: 24.31
GENERATED RESPONSE:
Okay, so I need to generate 10 examples of irony and non-ironic text. The user has already provided 5 examples for each, but I need to come up with the remaining ones. Let me think about how to approach this.

First, I should recall what irony is. Irony is a form of sarcasm where the meaning of the text is opposite to its literal or figurative meaning. It's often subtle and relies on ambiguity or surprise. So, for non-ironic texts, I need to avoid any situation where the meaning is reversed.

Let me start by brainstorming some common situations where irony isn't present. Maybe use simple sentences where the meaning is straightforward. For example, stating facts without any sarcasm.

Text: "The sun is shining bright." Label: non-irony.

Another one could be a question where the answer is obvious. "What's the capital of France?" Label: non-irony.

I should make sure each example is clear and doesn't have any hidden meanings. Maybe use contractions an

In [17]:
prompt = f"""\
You are an expert linguist and NLP practitioner specializing in irony detection. \
Your task is to generate **10 high-quality examples** of ironic and non-ironic statements, \
with **5 ironic** and **5 non-ironic** examples across different contexts.

### **Output Format (JSON)**
Return the final result into a valid JSON list in the following structure:

[
    {{"text": "{first_irony}", "label": "ironic"}},
    {{"text": "{first_non_irony}", "label": "non-ironic"}},
    ...
]
"""

response, delta_t = get_response(prompt, model, tokenizer, max_new_tokens=2048)

# add to log
synthetic_data = response2json(response)
log_synthetic_data(model_name, "baseline", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 14.05
GENERATED RESPONSE:
Alright, so the user has provided a query where they're asking for 10 examples of ironic and non-ironic statements, with exactly 5 of each. The output needs to be in JSON format. 

First, I need to understand what irony is. Irony occurs when there's a contradiction between what's expected and what actually happens. It's often due to the listener's interpretation of the words. So, the task is to generate statements where some are clearly ironic and others are not, across different contexts.

I'll start by brainstorming some ironic statements. Maybe something like a person with a broken leg being excited for the next 3 weeks. That's ironic because having a broken leg usually makes you feel worse, but the person is hoping to feel better. 

Next, a situation where a broken arm is causing a problem. For example, someone with broken arms is struggling to get up, which is ironic because it's a clear contradiction.

Then, a non-ironic example could be some

### Targeted synthetic data


In [18]:
prompt = f"""\
You are an expert linguist and NLP practitioner. \
List all key linguistic and semantic phenomena that must be considered for an irony detection model to perform accurately. \
Provide only the names of these phenomena as a structured list, without explanations. \
For example: sarcasm, negation, pragmatic inference, unexpected contrast. \
Return the list in a simple bullet-point format, each bullet point must start with a -.\
"""

response, delta_t_0 = get_response(prompt, model, tokenizer, max_new_tokens=2048)

TIME TAKEN: 20.16
GENERATED RESPONSE:
Okay, so I'm trying to figure out all the key linguistic and semantic phenomena that an irony detection model needs to consider. I'm not super familiar with all the terms, but I know irony is when something is ironic, meaning it's contradictory or surprising. So, I need to think about what makes something ironic.

First, I remember that sarcasm involves intentional misinterpretation or distortion of meaning. So, that's probably one. Then there's negation, which is when the opposite of something is implied. That makes sense because irony often involves a double negative or a reversal of expectations.

Another one is unexpected contrast. I think that's when something is contradictory but not obviously so because it's unexpected. Like, saying "I love you" when you're actually not that into someone. That's a classic example.

There's also the aspect of surprise. If someone says something that's not expected, that can be irony. Like, saying "I'm so hung

In [20]:
# Extract bullet points
bullets = re.findall(r"- (.+)", response)
bullets = [b.strip() for b in bullets]  # Remove leading/trailing whites

# Convert to CSV string
csv_string = ", ".join(bullets)

prompt = f"""\
Generate 10 realistic sentences illustrating irony detection examples involving {csv_string}. Ensure that:
- 5 sentences are **ironic**.
- 5 sentences are **non-ironic**.

### **Output Format (JSON)**
Return **only** a valid JSON list in the following structure:

[
    {{"text": "{first_irony}", "label": "ironic"}},
    {{"text": "{first_non_irony}", "label": "non-ironic"}},
    ...
]
"""

response, delta_t = get_response(prompt, model, tokenizer, max_new_tokens=2048)
delta_t += delta_t_0 # Add the time taken for the previous response

# add to log
synthetic_data = response2json(response)
log_synthetic_data(model_name, "targeted", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 16.95
GENERATED RESPONSE:
Alright, so the user wants me to generate 10 sentences that illustrate irony detection examples. They specified that 5 should be ironic and 5 non-ironic. Hmm, okay, I need to make sure I understand what irony means here. Irony is when the meaning of something is opposite to its actual sense, often due to the content. So, I need to create sentences where the irony is clear and intentional.

First, I'll brainstorm some situations where irony naturally comes into play. Maybe comparing something that's supposed to be positive to something that's negative. Like, a person saying they're happy when they're sick, or a situation where a positive action leads to a negative outcome. That should make it ironic.

For the ironic sentences, I can think of scenarios where the person is happy, but it's because they're in a situation where they're not supposed to be. For example, someone who's feeling good about themselves when they're sick, or someone who's upset w